## **Library Imports**

In [ ]:
# @markdown This cell installs and loads the required libraries and connects to Google Drive. Running this cell will ask you to authenticate using your google account credentials.

# connect to google sheets through a sheet URL
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

# install and load fuzzywuzzy
!pip install fuzzywuzzy[speedup]
from fuzzywuzzy import fuzz

# load other libraries
import pandas as pd
from datetime import datetime
import pytz
import time

#parallel processing
!pip install pandarallel
from pandarallel import pandarallel
pandarallel.initialize()

# clears the output of this cell block
from IPython.display import clear_output
clear_output()

## **Data Connections**

In [ ]:
# @markdown This cell connects to your analysis sheet. Paste the link here and run.
analysis_url = 'https://docs.google.com/spreadsheets/d/1R09CkSXedn6paLk8q3qUAYiCFp1g84K05Wu6JxfgDF0/edit#gid=0' #@param {type:"string"}
analysis_ws = 'Matching'
admin_ws = 'Admin'

In [ ]:
# @markdown This cell collects the key products and associated keyworks from MAIA.

# Connect to MAIA
maia_url = 'https://docs.google.com/spreadsheets/d/1zit8jJhj7ccbX9P4iiXaW3Fxs5ITrEaZ6UZCw9isEAQ/'       #id of MAIA spreadsheet with key products
maia_ws = 'DB_KeyProd'                #worksheet with list

# Import the MAIA data from google sheets
wb_maia = gc.open_by_url(maia_url)
sheet_maia = wb_maia.worksheet(maia_ws)
row = 5
col = 1
maia_df = pd.DataFrame(sheet_maia.get_all_values(),columns=sheet_maia.row_values(row)).iloc[row:].reset_index(drop=True)

#make df with key products, keywords
df_key = maia_df[["Key Product", "Keywords"]].reset_index(drop=True)
df_key.columns = ['key_product','keywords','keyword_count']

# split the keyword string to a list
df_key["keywords"]=df_key["keywords"].str.split(";")

# add the keyproduct name to the keyword list and remove blanks
def append_keyprod(x):
  x["keywords"].append(x["key_product"])
  x['keywords'] = list(filter(None, x['keywords']))
  return x

df_key = df_key.apply(append_keyprod,axis=1)

print("%s key products loaded" % df_key.key_product.count())

725 key products loaded


## **Algorithm Settings**

In [ ]:
# @markdown This cell sets some algorithm settings. Don't change these unless you know what you're doing.

# @markdown **Fuzzy logic algorithm:** 'simple', 'partial', 'sort', or 'set'. The 'set' algorithm performs the best
fw_routine = 'set' #@param ['simple','partial','sort','set']

# @markdown **Single pass matching:** If TRUE, objects with multiple perfect matches have the first one selected. If FALSE, they are matched again with another algorithm to choose the best one.
single_pass = 'False' #@param ['True', 'False']

# @markdown [Not implemented yet] **Rerun:** If TRUE, rerun everything. If FALSE, only entries that are new (don't have a match)
rerun = 'True' #@param ['True','False']


## **Functions**

In [ ]:
# @markdown This cell defines the necessary functions.

def run_fuzzywuzzy (keyproduct,item,method):
  try:
    scores = []
    for keyword in keyproduct["keywords"]:
      if method == 'simple':
        scores.append(fuzz.ratio(item, keyword))
      elif method == 'partial':
        scores.append(fuzz.partial_ratio(item, keyword))
      elif method == 'sort':
        scores.append(fuzz.token_sort_ratio(item, keyword))
      elif method == 'set':
        scores.append(fuzz.token_set_ratio(item, keyword))
    return max(scores)
  except:
    return 0

# match an individual object name to a keyproduct
def match_individual (item, df_key, method):
  df_scores = pd.Series(dtype='string')
  df_scores = df_key.apply(run_fuzzywuzzy,args = [item,method], axis = 1)
  return list(df_scores)

# use the single pass matching method
# if an item gets multiple 100% matches, only the first is selected
def single_pass_matching(x, df_key, method):
  # df_key: dataframe column of keyproducts
  # method: which fuzzy logic routine to run
  scores = match_individual(x["objects"], df_key, method)
  x['match'] = df_key["key_product"][scores.index(max(scores))]
  x['score'] = max(scores)
  x['keyword_count'] = df_key["keyword_count"][scores.index(max(scores))]
  return x

# use the multi pass matching method
# if an item gets multiple 100% matches, the fuzzy wuzzy'sort' method is used on the 100% matches
def double_pass_matching(x, df_key, method):
  # df_keyprod: dataframe column of keyproducts
  # method: fuzzywuzzy routine to match strings; 'simple', 'partial', 'sort', or 'set'

  first_scores = match_individual(x["objects"], df_key, fw_routine)  
  matches = []
  for i in range(len(first_scores)):
    if first_scores[i] == 100:
      matches.append(df_key.iloc[i])
  df_matches = pd.DataFrame(matches).reset_index(drop=True)
  
  if len(matches)>1:
    second_scores = match_individual(x['objects'],df_matches,'sort')
    result = [df_matches['key_product'][second_scores.index(max(second_scores))], 100, df_matches['keyword_count'][second_scores.index(max(second_scores))]]
  else:
    result = [df_key["key_product"][first_scores.index(max(first_scores))], max(first_scores), df_key["keyword_count"][first_scores.index(max(first_scores))]]
  x['match'] = result[0]
  x['score'] = result[1]
  x['keyword'] = result[2]
  return result

# initiates the correct matching function
def master_matching(df, df_key, method, single_pass):
  # df: df of procurement object names
  # df_key: df of key products+keywords
  # method: fuzzywuzzy routine to match strings; 'simple', 'partial', 'sort', or 'set'
  # single_pass: whether to run single_pass or double_pass
  
  if single_pass:
    df = df.parallel_apply(single_pass_matching, args=(df_key, fw_routine), axis=1)
  else:
    df = df.parallel_apply(double_pass_matching, args=(df_key, fw_routine), axis=1)
  return df

# function to bin the scores to get a summary
def bin_scores (df):
  my_bins = [0,20,40,60,80,100]
  my_labels = [1,2,3,4,5]

  df["bins"] = pd.cut(df.score, bins = my_bins, labels = my_labels)
  binned = df.groupby(['bins']).agg('count')['objects']
  binned = round(binned / binned.sum() * 100) 
  return binned

# output the data into the worksheet
def output_to_ws(data, column, ws, row):
  #data: df column
  #column: string of column index; 'A', 'B', 'C' etc.
  #ws: worksheet of interest

  cell_list = ws.range(column+str(row)+':'+column+str(len(data)+row-1))
  for i in range(len(cell_list)):
    try:
      cell_list[i].value = float(data[i])
    except:
      cell_list[i].value = data[i]
  ws.update_cells(cell_list)

# calculate and populate the admin sheet
def admin(start, end, output, wb):
  # collect admin info
  runtime = "%s sec" % (round(end - start,2))
  executiontime = datetime.now(pytz.timezone("CET")).strftime("%d/%m/%Y %H:%M")
  count = output.objects.count()
  binned_scores = bin_scores(output)
  # write admin info into spreadsheet
  sheet_admin = wb.worksheet(admin_ws)
  sheet_admin.update_cell(8,2, executiontime)
  sheet_admin.update_cell(9,2, runtime)
  sheet_admin.update_cell(10,2, float(count))
  # write summary into spreadsheet
  sheet_admin.update_cell(12,2, binned_scores[4])
  sheet_admin.update_cell(13,2, binned_scores[3])
  sheet_admin.update_cell(14,2, binned_scores[2])
  sheet_admin.update_cell(15,2, binned_scores[1])
  sheet_admin.update_cell(16,2, binned_scores[0])

  print ("Matching Done! Runtime: %s" % runtime)

## **Main**

In [ ]:
# @markdown This cell runs the matching and outputs the results. Runtime varies depending on the length of the procurement list and the average number of keywords per product.
# @markdown * 100 entries: 10 sec
# @markdown * 500 entries: 35 sec
# @markdown * 1000 entries: 71 sec

start = time.time()
# Import the procurement data from google sheets
wb = gc.open_by_url(analysis_url)
sheet = wb.worksheet(analysis_ws)
df_proc = pd.DataFrame(sheet.get_all_values(),columns=sheet.row_values(1)).iloc[1:].reset_index(drop=True)
df_proc.columns = ["objects","match","score","manual"]

# make guesses and add them to df_obj
output = master_matching(df_proc, df_key, fw_routine, single_pass)

# write data into spreadsheet
output_to_ws(output['match'], 'B', sheet, 2)
output_to_ws(output['score'], 'C', sheet, 2)

end = time.time()

# write run details to admin sheet
admin(start, end, output, wb)


Matching Done! Runtime: 7.7 sec
